In [1]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules
import pandas as pd
import numpy as np
import re

1. Use a livraria ‘mlxtend’ para extrair os padrões frequentes do dataset, aplicando o algoritmo FPGrowth. Considere um suporte mínimo de 60%. 

In [2]:
dataset = [['Leite', 'Cerveja', 'Pão'], 
 ['Leite', 'Cerveja', 'Frutas'], 
 ['Leite', 'Cerveja'], 
 ['Leite', 'Pão']]

In [3]:
te = TransactionEncoder()

df = pd.DataFrame(te.fit(dataset).transform(dataset), columns=te.columns_)
df

,Cerveja,Frutas,Leite,Pão
0,True,False,True,True
1,True,True,True,False
2,True,False,True,False
3,False,False,True,True


In [4]:
fpgrowth(df, min_support=0.6)

,support,itemsets
0,1.00,(2)
1,0.75,(0)
2,0.75,"(0, 2)"


2. Analise o dataset ‘ComprasOnline.xlsx’. 
a) Detete problemas nos dados, como por exemplo existência de NAs e faça um pré 
processamento de limpeza nos mesmos.

In [5]:
df = pd.read_excel("ComprasOnline.xlsx")
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
2,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
3,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,2011-12-09 12:50:00,3.75,12680.0,France
541905,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541906,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23256,CHILDRENS CUTLERY SPACEBOY,4,2011-12-09 12:50:00,4.15,12680.0,France


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [7]:
# Remover faturas (invoice) com letras A e C
df_no_alpha_invoice = df[df["InvoiceNo"].apply(lambda x: str(x).isnumeric())]
df_no_alpha_invoice.info()

<class 'pandas.core.frame.DataFrame'>
Index: 532618 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    532618 non-null  object        
 1   StockCode    532618 non-null  object        
 2   Description  531164 non-null  object        
 3   Quantity     532618 non-null  int64         
 4   InvoiceDate  532618 non-null  datetime64[ns]
 5   UnitPrice    532618 non-null  float64       
 6   CustomerID   397924 non-null  float64       
 7   Country      532618 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.6+ MB


In [8]:
# Converter stockcode para string e depois remover todas as letras
df_stockcode = df_no_alpha_invoice.copy()
df_stockcode["StockCode"] = df_stockcode["StockCode"].apply(lambda x: re.sub(r"[a-zA-Z_]+", "", str(x)))

# Remover linhas com stockcode em branco
df_stockcode = df_stockcode[df_stockcode["StockCode"].apply(lambda x: re.match(r"\s*$", str(x)) == None)]

In [9]:
# Fazer a pivot table de forma a que somemos todas as quantidades entre um cliente e um item
df_france = df_stockcode[df_stockcode["Country"] == "France"]
df_france_reduced = pd.pivot_table(df_stockcode, values = "Quantity", index = "InvoiceNo", columns="StockCode", aggfunc = "sum", fill_value=0)

In [10]:
# Codificar com True e False, se a quantidade for positiva fica True se não false
df_aggr = df_france_reduced.map(lambda x: True if x > 0 else False)

In [11]:
# Calcular os padrões frequentes utilizando o apriori, temos de colocar um suporte mínimo de 0.1 para obter algum padrão frequente
# Tem de ser um valor baixo porque há uma elevada quantidade de dados
# O ColNames usa o StockCode para identificar o item dentro de cada itemset
padroes_freq = apriori(df_aggr, min_support=0.03, use_colnames=True)

In [12]:
# Criar as regras de associação usando a avaliação por Lift, ou seja, se lift > 1 a correlação é positiva entre o item set antecedente
# e o consequente, logo Se antecedente então consequente (é uma implicação)
regras = association_rules(padroes_freq, metric="lift", min_threshold=1.)
regras

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(20725),(85099),0.071523,0.121521,0.031808,0.444728,3.659685,0.023117,1.582071,0.782736
1,(85099),(20725),0.121521,0.071523,0.031808,0.261753,3.659685,0.023117,1.257677,0.827285
2,(85099),(21931),0.121521,0.054111,0.039623,0.326062,6.025821,0.033048,1.403526,0.949422
3,(21931),(85099),0.054111,0.121521,0.039623,0.732264,6.025821,0.033048,3.281133,0.881760
4,(85099),(22386),0.121521,0.055665,0.043828,0.360662,6.479182,0.037064,1.477051,0.962641
5,(22386),(85099),0.055665,0.121521,0.043828,0.787356,6.479182,0.037064,4.131226,0.895508
6,(85099),(22411),0.121521,0.053700,0.038024,0.312900,5.826856,0.031498,1.377237,0.942972
7,(22411),(85099),0.053700,0.121521,0.038024,0.708085,5.826856,0.031498,3.009367,0.875389
8,(22697),(22699),0.046387,0.048718,0.035099,0.756650,15.531205,0.032839,3.909114,0.981125
9,(22699),(22697),0.048718,0.046387,0.035099,0.720450,15.531205,0.032839,3.411246,0.983529


Na linha 6 tem se um item antecedente com suporte 0.12 e o consequente com 0.05, suportes baixos. A confiança da regra também não é alta, com apenas 0.31 e o suporte ainda mais pequeno com 0.03. Porém o Lift é de 5.8 sendo > 1. Ou seja, a regra encontra um padrão, em que se o antecedente acontece o consequente também vai acontecer, apesar do suporte a confiança dizer que é improvável, o lift permite avaliar regras "estranhas" mas úteis. 